There were some transactions not scoped on.
This is because the last line

In [20]:
from src.bank_transaction_reader import BankTransactionProcesser as btp
import re
processor = btp()
txt = processor.extract_text_from_pdf('bank-statement/Statement_2023_7.pdf')
pattern = r'(?<=Your Transactions\n)(.*?)(?=\(Continued on next page\)|Transaction types)'
tb = re.findall(
    pattern,
    txt,
    re.DOTALL
    )
print(tb[7])

Column
Date.Column
Description.Column
Type.Column
Money In (£).Column
Money Out (£).Column
Balance (£).
Date
28 Jul 23.Description
COSTA COFFEE 43011.Type
DEB.Money In (£)
blank.Money Out (£)
5.85.Balance (£)
2,690.75.
Date
28 Jul 23.Description
MARKS&SPENCER PLC.Type
DEB.Money In (£)
blank.Money Out (£)
10.05.Balance (£)
2,680.70.
Date
31 Jul 23.Description
STGCOACH/CTYLINK.Type
DEB.Money In (£)
blank.Money Out (£)
2.00.Balance (£)
2,678.70.
Date
31 Jul 23.Description
STGCOACH/CTYLINK.Type
DEB.Money In (£)
blank.Money Out (£)
2.00.Balance (£)
2,676.70.
Date
31 Jul 23.Description
PRET A MANGER.Type
DEB.Money In (£)
blank.Money Out (£)
8.65.Balance (£)
2,668.05.
Date
31 Jul 23.Description
PAYPAL *LINKEDIN.Type
DEB.Money In (£)
blank.Money Out (£)
21.87.Balance (£)
2,646.18.
Date
31 Jul 23.Description
Zettle_*Sundays.Type
DEB.Money In (£)
blank.Money Out (£)
6.95.Balance (£)
2,639.23.
Date
31 Jul 23.Description
MARKS&SPENCER PLC.Type
DEB.Money In (£)
blank.Money Out (£)
8.45.Balance (£)


In [1]:
# import pandas as pd
# from urllib.request import urlopen

# url = 'https://raw.githubusercontent.com/resbaz/r-novice-gapminder-files/master/data/gapminder-FiveYearData.csv'
# response = urlopen(url)

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
df=pd.read_parquet('data/bank-data.parquet')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575 entries, 0 to 574
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           575 non-null    datetime64[ns]
 1   Description    575 non-null    object        
 2   Type           575 non-null    object        
 3   Money In (£)   575 non-null    float64       
 4   Money Out (£)  575 non-null    float64       
 5   Balance (£)    575 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 27.1+ KB


In [204]:
px.scatter(
    df,x='Date',y='Money Out (£)'
)

In [190]:
fd=(df
    .assign(
        Day = lambda df: df.Date,
        WeekDay= lambda df: df.Date.dt.day_name(),
        Week=lambda df: df.Date.dt.week
    )
    .groupby(['Day','Week','WeekDay'])
    .agg(
        {
            'Money Out (£)':'sum',
            'Money In (£)':'sum'
         }
    )
    .reset_index()
)
fd

/var/folders/r5/1cdq52mn21zdnqzl0fvp44zw0000gn/T/ipykernel_4044/3719523316.py:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



,Day,Week,WeekDay,Money Out (£),Money In (£)
0,2023-06-01,22,Thursday,52.0,0.0
1,2023-06-02,22,Friday,55.0,0.0
2,2023-06-05,23,Monday,82.0,0.0
3,2023-06-06,23,Tuesday,32.0,0.0
4,2023-06-07,23,Wednesday,9.0,0.0
...,...,...,...,...,...
81,2023-09-25,39,Monday,233.0,0.0
82,2023-09-26,39,Tuesday,50.0,0.0
83,2023-09-27,39,Wednesday,9.0,0.0
84,2023-09-28,39,Thursday,19.0,0.0


In [191]:
import ipywidgets as widgets
fdm = fd.pivot(index='Week',columns='WeekDay')['Money Out (£)'].fillna(0)
fdm

WeekDay,Friday,Monday,Thursday,Tuesday,Wednesday
Week,,,,,
22,55.0,0.0,52.0,0.0,0.0
23,15.0,82.0,4.0,32.0,9.0
24,23.0,493.0,17.0,223.0,38.0
25,91.0,64.0,17.0,57.0,2.0
26,22.0,823.0,13.0,19.0,29.0
27,48.0,474.0,20.0,305.0,45.0
28,27.0,284.0,14.0,26.0,31.0
29,46.0,96.0,104.0,19.0,24.0
30,20.0,171.0,17.0,10.0,47.0


In [196]:
fig = px.imshow(
    fdm,
    color_continuous_scale='RdBu_r',
    origin='lower',aspect='equal',
)
fig.update_xaxes(side='top')
fig.show()


In [218]:
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
fig=go.Figure(data=go.Heatmap(
    x=fdm.columns,
    y=fdm.index,
    z=fdm.fillna(0),
    hovertemplate = "weekday: %{x} <br>" + "week: %{y} <br>" + "Spending: %{z}",
    colorscale='hot',
    colorbar=dict(
        title='Spending'
    )   
))
fig.update_xaxes(side='top')

In [193]:
px.colors.sequential.Cividis

['#00224e',
 '#123570',
 '#3b496c',
 '#575d6d',
 '#707173',
 '#8a8678',
 '#a59c74',
 '#c3b369',
 '#e1cc55',
 '#fee838']

In [188]:
pal=sns.color_palette('crest', as_cmap=True)

## Test Object Instances

In [192]:
from importlib import reload
import src
import pandas as pd
from src.bank_transaction_analyser import BankTransactionAnalyser as bta
from src.bank_transaction_reader import BankTransactionReader as btr
%config InlineBackend.figure_format = 'retina'

In [56]:
files = [
    'bank-statement/Statement_2023_6.pdf',
    'bank-statement/Statement_2023_7.pdf',
    'bank-statement/Statement_2023_8.pdf',
    'bank-statement/Statement_2023_9.pdf'
]

df = btr.read(files)

Read additional 7 tables.Total 7 tables populated.
Read additional 8 tables.Total 15 tables populated.
Read additional 8 tables.Total 23 tables populated.
Read additional 7 tables.Total 30 tables populated.


In [163]:
p = 'Month'
l=['Category']
l + [p]

['Category', 'Month']

In [235]:
reload(src.bank_transaction_analyser)
from src.bank_transaction_analyser import BankTransactionAnalyser as bta
import re
banker = bta(df)
pd.read_parquet('data/bank-data.parquet').shape[0] == banker.data.shape[0]
# banker.plot_spending_period('Month')
banker.plotly_spending_period('Month','bar',darkmode=True)

data is valid
initialized


In [234]:
import plotly.express as px
px.colors.qualitative.Vivid

['rgb(229, 134, 6)',
 'rgb(93, 105, 177)',
 'rgb(82, 188, 163)',
 'rgb(153, 201, 69)',
 'rgb(204, 97, 176)',
 'rgb(36, 121, 108)',
 'rgb(218, 165, 27)',
 'rgb(47, 138, 196)',
 'rgb(118, 78, 159)',
 'rgb(237, 100, 90)',
 'rgb(165, 170, 153)']